In [5]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms


# ✅ Step 1: Convert CSV Image Paths to Absolute Paths
data_dir = r"C:\Users\abdal\AI5\training_data"
csv_path = os.path.join(data_dir, "training_data.csv")

# Load CSV
df = pd.read_csv(csv_path, header=None)
df.columns = ["image_filename", "label", "centroid_x"]  # Rename columns

# ✅ Fix incorrect paths: Ensure subfolder (left, right, etc.) is included
df["image_filename"] = df.apply(lambda row: os.path.abspath(os.path.join(data_dir, row["label"], os.path.basename(row["image_filename"]))), axis=1)

# Save the updated CSV
updated_csv_path = os.path.join(data_dir, "training_data_absolute.csv")
df.to_csv(updated_csv_path, index=False)

print("✅ CSV Updated with Correct Absolute Paths")

# ✅ Step 2: Dataset Class
label_mapping = {
    "straight": 0,
    "left": 1,
    "right": 2,
    "off_line": 3
}

class LineFollowingDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]["image_filename"]
        label_str = self.data.iloc[idx]["label"]
        label = label_mapping.get(label_str, -1)

        if not os.path.exists(img_path):
            print(f"⚠️ Missing image: {img_path}")
            return None, None

        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"⚠️ Could not open image file: {img_path}")
            return None, None

        image = cv2.resize(image, (128, 128)) / 255.0  # Normalize
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add channel dim

        return image, label

# ✅ Step 3: Load Dataset and DataLoader
dataset = LineFollowingDataset(updated_csv_path)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# ✅ Step 4: Define CNN Model
class LineFollowerCNN(nn.Module):
    def __init__(self):
        super(LineFollowerCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(32 * 32 * 32, 128)  # 32x32 after pooling twice
        self.fc2 = nn.Linear(128, 4)  # 4 output classes (steering commands)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)  # No activation function here (CrossEntropyLoss applies softmax internally)
        return x

# ✅ Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LineFollowerCNN().to(device)

criterion = nn.CrossEntropyLoss()  # Classification loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("✅ CNN Model Initialized!")

# ✅ Training Parameters
num_epochs = 10  # Adjust as needed

# ✅ Training Loop
for epoch in range(num_epochs):
    total_loss = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

# ✅ Save the trained model
torch.save(model.state_dict(), "line_follower_cnn3.pth")
print("✅ Model training complete and saved as 'line_follower_cnn.pth'.")

# ✅ Load Trained Model for Evaluation
def load_trained_model():
    """Loads the trained CNN model for steering predictions."""
    try:
        model = LineFollowerCNN().to(device)
        model.load_state_dict(torch.load("line_follower_cnn3.pth", map_location=device))
        model.eval()
        print("✅ Model Loaded Successfully!")
        return model
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None

# ✅ Load the trained model
model = load_trained_model()

# ✅ Evaluation Mode
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"✅ Model Accuracy on Test Data: {accuracy:.2f}%")



✅ CSV Updated with Correct Absolute Paths
✅ CNN Model Initialized!
Epoch [1/10], Loss: 1.3944
Epoch [2/10], Loss: 0.9915
Epoch [3/10], Loss: 0.7040
Epoch [4/10], Loss: 0.4571
Epoch [5/10], Loss: 0.3034
Epoch [6/10], Loss: 0.2133
Epoch [7/10], Loss: 0.1599
Epoch [8/10], Loss: 0.1231
Epoch [9/10], Loss: 0.0973
Epoch [10/10], Loss: 0.0756
✅ Model training complete and saved as 'line_follower_cnn.pth'.
✅ Model Loaded Successfully!
✅ Model Accuracy on Test Data: 100.00%


C:\Users\abdal\AppData\Local\Temp\ipykernel_22124\1982063351.py:126: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("line_follower_cnn3.pth",

In [4]:
import pandas as pd

csv_path = r"C:\Users\abdal\AI5\training_data\training_data_absolute.csv"
df = pd.read_csv(csv_path)

# Count the number of occurrences for each class
print(df['label'].value_counts())


label
right       8
left        8
straight    2
off_line    2
Name: count, dtype: int64
